## Comparing v1(persona) vs v2(no persona) condition

In [ ]:
import pandas as pd

df = pd.read_csv("/home/ubuntu/eri/scripts/preds_q=-1_lang=en_country=us_temp=0.3_maxt=40_n=1_v1_fewshot=0_pv1.csv")
df.to_csv("q=-1_country=us_lang=en_model=Llama-2-13b-chat-hf_eval=mv_sample_pv1.csv",sep=";", index=False)


In [ ]:
import  pandas as pd
# persona 
df_v1 = pd.read_csv("/home/ubuntu/eri/scripts/preds_q=-1_lang=en_country=us_temp=0.3_maxt=40_n=1_v1_fewshot=0_pv1.csv", sep=)
# no persona
df_v2 = pd.read_csv("/home/ubuntu/eri/scripts/q=-1_country=us_lang=en_model=Llama-2-13b-chat-hf_eval=mv_sample_pv2.csv")



In [ ]:
key_columns = [
    
]

In [3]:
import pandas as pd


In [6]:
persona_path = "align_q=2_lang=en_country=us_temp=0.3_maxt=40_n_gen=1.csv"
nopersona_path = "align_q=2_lang=en_country=us_temp=0.3_maxt=40_n_gen=1_no-persona.csv"

answer_col = "model-answer"

persona_cols = [
    "persona.region",
    "persona.sex",
    "persona.age",
    "persona.marital_status",
    "persona.education",
    "persona.social_class",
]
persona_cols


['persona.region',
 'persona.sex',
 'persona.age',
 'persona.marital_status',
 'persona.education',
 'persona.social_class']

In [35]:
df_persona = pd.read_csv(persona_path)
df_nopersona = pd.read_csv(nopersona_path)

print(f"Persona file shape: {df_persona.shape}")
print (f"No-persona file shape: {df_nopersona.shape}")

#display(df_persona.head())
#display(df_nopersona.head())


Persona file shape: (2246, 19)
No-persona file shape: (2250, 19)


In [36]:
# Merge on  person attributes
merged = df_persona.merge(
    df_nopersona,
    on=persona_cols,
    suffixes=("_persona", "_nopersona"),
)

#print(f"Merged shape: {merged.shape}")
#display(merged.head())

In [33]:
persona_raw = merged[f"{answer_col}_persona"]
nopersona_raw = merged[f"{answer_col}_nopersona"]

persona_num = pd.to_numeric(persona_raw, errors="coerce")
nopersona_num = pd.to_numeric(nopersona_raw, errors="coerce")

valid_mask = persona_num.notna() & nopersona_num.notna()

persona_answers = persona_num[valid_mask]
nopersona_answers = nopersona_num[valid_mask]

num_items = len(persona_answers)
print(f"Number of valid numeric pairs: {num_items}")

if num_items == 0:
    raise SystemExit("No valid numeric pairs, cannot compute kappa")

pd.DataFrame({
    "answer_nopersona": nopersona_answers.head(5).tolist(),
    "answer_persona": persona_answers.head(5).tolist(),

})

Number of valid numeric pairs: 2246


,answer_nopersona,answer_persona
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [34]:
# answers distributions


persona_counts = persona_answers.value_counts().sort_index()
nopersona_counts = nopersona_answers.value_counts().sort_index()

print("Counts persona:")
display(persona_counts)

print("\nCounts no-persona:")
display(nopersona_counts)

Counts persona:


model-answer_persona
1    2237
2       8
5       1
Name: count, dtype: int64


Counts no-persona:


model-answer_nopersona
1    2246
Name: count, dtype: int64

In [17]:
# confusion matrix
confusion_matrix = pd.crosstab(
    nopersona_answers, #rows
    persona_answers #columns
)

print("Confusion matrix (rows no-persona, cols: persona):")
display(confusion_matrix)

Confusion matrix (rows no-persona, cols: persona):


model-answer_persona,1,2,5
model-answer_nopersona,,,
1,2237,8,1


In [18]:

#Observed agreement Po
# Po = proportion of items where both answers are exactly equal

num_matches = (persona_answers == nopersona_answers).sum()
Po = num_matches/ num_items
print(f" Number of exact matche: {num_matches}")
print(f"Po (observed agreement): {Po}")

 Number of exact matche: 2237
Po (observed agreement): 0.9959928762243989


In [29]:
# expected agreetment
persona_prob = persona_counts/num_items
nopersona_prob = nopersona_counts / num_items

all_labels = sorted(set(persona_prob.index) | set(nopersona_prob.index))

Pe = 0.0
for label in all_labels:
    p_persona = persona_prob.get(label, 0.0)
    
    p_nopersona = nopersona_prob.get(label, 0.0)
    Pe += p_persona*p_nopersona

print(f"Pe: {Pe}")

Pe: 0.9959928762243989


In [ ]:
#   kappa = (Po - Pe) / (1 - Pe)
#   -  1.0  : perfect agreement
#   -  0.0  : no better than chance
#   - <0.0  : worse than chance

if  Pe == 1.0:
    kappa = float("nan") # only one category
else: 
    kappa = (Po -Pe)/ (1 - Pe)

print(f"K: ")

K: 0.0


In [ ]:
import pandas as pd

def kappa_calculation(name, array_1, array_2):
    persona_answers = pd.Series(array_1)
    nopersona_answers = pd.Series(array_2)

    num_items = len(persona_answers)
    print(f"Numero de Items: {num_items}")

    # Distribuciones
    persona_counts = persona_answers.value_counts().sort_index()
    nopersona_counts = nopersona_answers.value_counts().sort_index()

    print("Counts persona:")
    print(persona_counts)
    print("Counts nopersona")
    print(nopersona_counts)

    # Po (acuerdo observado)
    num_matches = (persona_answers == nopersona_answers).sum()
    Po = num_matches/num_items

    # Pe (acuerdo esperado)
    persona_prob = persona_counts / num_items
    nopersona_prob = nopersona_counts / num_items

    all_labels = sorted(set(persona_prob.index)|set(nopersona_prob.index))

    Pe = 0.0
    for label in all_labels:
        p_persona = persona_prob.get(label, 0.0)
        p_nopersona = nopersona_prob.get(label, 0.0)
        Pe += p_persona*p_nopersona
    
    # Kappa
    if Pe == 1.0:
        kappa = float("nan")
    else:
        kappa = (Po -Pe)/(1-Pe)

    print(f"Po= {Po}")
    print(f"Pe= {Pe}")
    print(f"kappa= {kappa}")
    print()
    
















In [42]:
array_1 = ["agree", "agree", "agree","agree","agree"]
array_2 = ["agree", "agree", "agree","agree","agree"]
kappa_calculation("CASE 1", array_1, array_2)

Numero de Items: 5
Counts persona:
agree    5
Name: count, dtype: int64
Counts nopersona
agree    5
Name: count, dtype: int64
Po= 1.0
Pe= 1.0
kappa= nan

